# Start hardware on board power up

This example shows you how to create hardware that starts when the board powers up over a serial connection. We will use the onboard LED. The steps are nearly identical over a WiFi connection. See start_webserver_over_serial.ipynb for details of starting a WiFi connection.

## Import packages

In [2]:
import sys
import os
import time

# Get the current working directory (the directory where the notebook is located)
notebook_dir = os.getcwd()

# Add the 'source' folder to the path
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..', 'source')))

# Now you can import from 'source'
from serial_controller import SerialController
from serial_command_sender import SerialCommandSender
from web_controller import WebController
from web_command_sender import WebCommandSender


## Initialize the Serial Command Sender and configure LED to be ON immediately after the board starts up.

Note: make sure nothing else is holding the serial line (for instance, a micropico VSC project). You may need to disconnect and reconnect the Pico's USB serial link.

In [ ]:
SERIAL_PORT = "COM6" # Change as needed
# Initialize the serial controller with the appropriate port. This must be done in every cell in jupyter notebooks to avoid holding the serial line. There is no such restriction in a python script.
serial_controller = SerialController(port=SERIAL_PORT, debug=False)  # Adjust port as needed
serial_controller.connect()
# Initialize the command sender
serial_command_sender = SerialCommandSender(serial_controller)

# Create a GPIO hardware, this case we'll use the LED
# Currently, there's a bug requireing the user to apply hardware settings for start on init to be applied.
# In other words, creating with start_on_init = True will not work, but applying the settings after creation will.
settings = {
    'pin_number': "LED", # Use the built-in LED
    'mode': 'OUT', # Set the pin as an output
    'value': 1, # Set the initial value to 1 (turn on the LED)
    'start_on_init': True # Start the LED on initialization
}
print(serial_command_sender.send_command('create', 'gpio', settings, "persistent_gpio"))
print(serial_command_sender.send_command('apply_hardware_settings', "persistent_gpio", {'start_on_init:': True}))

# Start the GPIO hardware, this assigns to settings to an actual component on the board
print(serial_command_sender.send_command('start', 'persistent_gpio'))

# Save the current configuration
print("Saving current configuration...")
print(serial_command_sender.send_command('save_config')) # Now whenever the RP Pico W is powered on, the LED will be on automatically

# Get the entire configuration
print("Saved configuration:")
response = serial_command_sender.send_command('get_all_config')
print(response)

serial_controller.close() #Serial controller must be closed in each cell in jupyter notebooks to avoid holding the serial line. There is no such restriction in a python script.


# Test the configuration by restarting the board manually. I.e. unplug and plug the board back in. The LED should be on when the board is powered on.


Connected to COM6
Command sent: run_command("create","gpio", {"pin_number": "LED", "mode": "OUT", "value": 1, "start_on_init": True}, "persistent_gpio")
Created GPIO hardware with ID persistent_gpio.
Command sent: run_command("apply_hardware_settings","persistent_gpio", {"start_on_init:": True})
Settings applied to hardware ID persistent_gpio: {'start_on_init:': True}
Command sent: run_command("start","persistent_gpio")
Hardware with ID persistent_gpio started.
Saving current configuration...
Command sent: run_command("save_config")
Configuration saved.
Saved configuration:
Command sent: run_command("get_all_config")
All Configuration: {'hardware': {'persistent_gpio': {'settings': {'value': 1, 'hardware_id': 'persistent_gpio', 'pin_number': 'LED', 'start_on_init': True, 'mode': 'OUT'}, 'type': 'GPIOHardware'}}}
Connection to COM6 closed.


## Remove the LED from startup
To stop the LED from being on on startup you have 3 options:
1. Set `hardware.persistent_gpio.value = 0` in the config, then save the config. In this case the LED will start in the off state.
2. Set `hardware.persistent_gpio.start_on_init = False` in the config the config. In this case you will have to send the start command to turn the LED on again. 
3. Delete `hardware.persistent_gpio` in the config. In this case, you will have to create a new gpio hardware to turn the LED on.

Below shows the 3 options, however option 3 will override the first 2 as it is done last.

In [ ]:
# Initialize the serial controller with the appropriate port. This must be done in every cell in jupyter notebooks to avoid holding the serial line. There is no such restriction in a python script.
serial_controller = SerialController(port=SERIAL_PORT, debug=False)  # Adjust port as needed
serial_controller.connect()
# Initialize the command sender
serial_command_sender = SerialCommandSender(serial_controller)

# Option 1: Turn off the LED
new_settings_1 = {
    'value': 0
}
print(serial_command_sender.send_command('apply_hardware_settings', 'persistent_gpio', new_settings_1))
print(serial_command_sender.send_command('save_config')) # Save the new configuration

# Option 2: Set start_on_init to False
new_settings_2 = {
    'start_on_init': False
}
print(serial_command_sender.send_command('apply_hardware_settings', 'persistent_gpio', new_settings_2))
print(serial_command_sender.send_command('save_config')) # Save the new configuration

# Option 3: Remove the GPIO hardware
print(serial_command_sender.send_command('delete', 'persistent_gpio'))
print(serial_command_sender.send_command('save_config')) # Save the new configuration

# Get the entire configuration
print("Saved configuration:")
response = serial_command_sender.send_command('get_all_config')
print(response)


serial_controller.close() #Serial controller must be closed in each cell in jupyter notebooks to avoid holding the serial line. There is no such restriction in a python script.



Connected to COM6
Command sent: run_command("apply_hardware_settings","persistent_gpio", {"value": 0})
Settings applied to hardware ID persistent_gpio: {'value': 0}
Command sent: run_command("save_config")
Configuration saved.
Command sent: run_command("apply_hardware_settings","persistent_gpio", {"start_on_init": False})
Settings applied to hardware ID persistent_gpio: {'start_on_init': False}
Command sent: run_command("save_config")
Configuration saved.
Command sent: run_command("delete","persistent_gpio")
Hardware with ID persistent_gpio deleted.
Command sent: run_command("save_config")
Configuration saved.
Saved configuration:
Command sent: run_command("get_all_config")
All Configuration: {'hardware': {}}
Connection to COM6 closed.
